In [250]:
%matplotlib inline
%pwd

'/home/gabe/work/fast-ai/nbs'

In [3]:
%%html
<style>
  .end_space {
      min-height: 1000px;
  }
  .container {
      width: 100%;
  }
</style

In [4]:
import sys
import os
from importlib import reload

sys.path.append('../src')

In [5]:
DATA_DIR = '../data/dogbreed'
SAMPLE_DIR = os.path.join(DATA_DIR, 'sample')
BATCH_SIZE = 64

In [257]:
import utils.dogbreed
reload(utils.dogbreed)

import utils.trainhelper
reload(utils.trainhelper)

from utils.dogbreed import *
from utils.trainhelper import get_batches, save_model, read_model, get_classes
from utils.utils import save_array, load_array

from models.vgg16 import VGG16
from models.vgg19 import VGG19

In [258]:
setup_train_data(DATA_DIR)

train ../data/dogbreed/train/00ca18751837cd6a22813f8e221f7819.jpg ../data/dogbreed/train/affenpinscher/00ca18751837cd6a22813f8e221f7819.jpg
train ../data/dogbreed/train/01268f0007876a692907bda89468184c.jpg ../data/dogbreed/train/affenpinscher/01268f0007876a692907bda89468184c.jpg
train ../data/dogbreed/train/0162107acd8f2588c0944b791d61bb0c.jpg ../data/dogbreed/train/affenpinscher/0162107acd8f2588c0944b791d61bb0c.jpg
train ../data/dogbreed/train/040fef64640b89c53f161b6c5215b78b.jpg ../data/dogbreed/train/affenpinscher/040fef64640b89c53f161b6c5215b78b.jpg
train ../data/dogbreed/train/0484a6cc686a07e1edacd0fb44b1d965.jpg ../data/dogbreed/train/affenpinscher/0484a6cc686a07e1edacd0fb44b1d965.jpg
train ../data/dogbreed/train/100f347ec78a42a9e7c2418e4beb3f6a.jpg ../data/dogbreed/train/affenpinscher/100f347ec78a42a9e7c2418e4beb3f6a.jpg
train ../data/dogbreed/train/106d7e0df869e7ba9f6d16c5f77ddfb8.jpg ../data/dogbreed/train/affenpinscher/106d7e0df869e7ba9f6d16c5f77ddfb8.jpg
train ../data/dogbre

train ../data/dogbreed/train/bf25cf910748150b47bbf727a0dd6ef7.jpg ../data/dogbreed/train/airedale/bf25cf910748150b47bbf727a0dd6ef7.jpg
train ../data/dogbreed/train/c02f83947002646c705681ef6433294a.jpg ../data/dogbreed/train/airedale/c02f83947002646c705681ef6433294a.jpg
train ../data/dogbreed/train/c1a5f4fb96ec93193d7d105bcf62fd45.jpg ../data/dogbreed/train/airedale/c1a5f4fb96ec93193d7d105bcf62fd45.jpg
train ../data/dogbreed/train/c3912710a89f0e88320b8fe07578a972.jpg ../data/dogbreed/train/airedale/c3912710a89f0e88320b8fe07578a972.jpg
train ../data/dogbreed/train/c3ffe489a6dd65d192c0d839ded36e94.jpg ../data/dogbreed/train/airedale/c3ffe489a6dd65d192c0d839ded36e94.jpg
train ../data/dogbreed/train/c44069838d7f06552dbbd8dea615d901.jpg ../data/dogbreed/train/airedale/c44069838d7f06552dbbd8dea615d901.jpg
train ../data/dogbreed/train/c5d485d5c12306d060bba17295d3d435.jpg ../data/dogbreed/train/airedale/c5d485d5c12306d060bba17295d3d435.jpg
train ../data/dogbreed/train/c8d4d8654df681140522be0f3b

train ../data/dogbreed/train/7bea6cd93e219ca2419f999001ecb27a.jpg ../data/dogbreed/train/beagle/7bea6cd93e219ca2419f999001ecb27a.jpg
train ../data/dogbreed/train/8436be99589db6a99cfac1b894421ea6.jpg ../data/dogbreed/train/beagle/8436be99589db6a99cfac1b894421ea6.jpg
train ../data/dogbreed/train/8687dd4fbc2b4ec87aef79775c177c27.jpg ../data/dogbreed/train/beagle/8687dd4fbc2b4ec87aef79775c177c27.jpg
train ../data/dogbreed/train/86f0e4abee677119258764eadc368b9d.jpg ../data/dogbreed/train/beagle/86f0e4abee677119258764eadc368b9d.jpg
train ../data/dogbreed/train/88435e91be34908f134a5ef10ac80dd0.jpg ../data/dogbreed/train/beagle/88435e91be34908f134a5ef10ac80dd0.jpg
train ../data/dogbreed/train/8847eac09561675bf7e3d6d9632b8865.jpg ../data/dogbreed/train/beagle/8847eac09561675bf7e3d6d9632b8865.jpg
train ../data/dogbreed/train/8c061cc94ba2801a2c5f85af5653ee58.jpg ../data/dogbreed/train/beagle/8c061cc94ba2801a2c5f85af5653ee58.jpg
train ../data/dogbreed/train/8caee24979a5ad8887ceba75d93f0ab6.jpg ../

train ../data/dogbreed/train/04d50b0dce8b8ee9544d9602df85f722.jpg ../data/dogbreed/train/bluetick/04d50b0dce8b8ee9544d9602df85f722.jpg
train ../data/dogbreed/train/05a747c5853defa2420b976a9c0918a2.jpg ../data/dogbreed/train/bluetick/05a747c5853defa2420b976a9c0918a2.jpg
train ../data/dogbreed/train/09aec2a169942b17d88b4b5f1bba5886.jpg ../data/dogbreed/train/bluetick/09aec2a169942b17d88b4b5f1bba5886.jpg
train ../data/dogbreed/train/0dcb8150908bbb7b7fff1b9d41a358ba.jpg ../data/dogbreed/train/bluetick/0dcb8150908bbb7b7fff1b9d41a358ba.jpg
train ../data/dogbreed/train/0e79be614f12deb4f7cae18614b7391b.jpg ../data/dogbreed/train/bluetick/0e79be614f12deb4f7cae18614b7391b.jpg
train ../data/dogbreed/train/1015e4eba660def32e7e52b89fe3a9c6.jpg ../data/dogbreed/train/bluetick/1015e4eba660def32e7e52b89fe3a9c6.jpg
train ../data/dogbreed/train/1e1650836e5d0dc6b49b2d3c91b36d59.jpg ../data/dogbreed/train/bluetick/1e1650836e5d0dc6b49b2d3c91b36d59.jpg
train ../data/dogbreed/train/202bb947018debda922ef6af3c

train ../data/dogbreed/train/304e2211dbfa90d7d5818a53b101f397.jpg ../data/dogbreed/train/bouvier_des_flandres/304e2211dbfa90d7d5818a53b101f397.jpg
train ../data/dogbreed/train/32087d5446dfe75c2c25bfb2110b2f09.jpg ../data/dogbreed/train/bouvier_des_flandres/32087d5446dfe75c2c25bfb2110b2f09.jpg
train ../data/dogbreed/train/37e96534152743e871174b98424ce638.jpg ../data/dogbreed/train/bouvier_des_flandres/37e96534152743e871174b98424ce638.jpg
train ../data/dogbreed/train/3a06c2bc81ca093c59de497d7bbef5aa.jpg ../data/dogbreed/train/bouvier_des_flandres/3a06c2bc81ca093c59de497d7bbef5aa.jpg
train ../data/dogbreed/train/3af48e9cc72eb4faf15eb348c741a676.jpg ../data/dogbreed/train/bouvier_des_flandres/3af48e9cc72eb4faf15eb348c741a676.jpg
train ../data/dogbreed/train/3b68257442e2a4486c4bc240aaddc8ba.jpg ../data/dogbreed/train/bouvier_des_flandres/3b68257442e2a4486c4bc240aaddc8ba.jpg
train ../data/dogbreed/train/3c6a53595e1cd63e63fdc166e4223606.jpg ../data/dogbreed/train/bouvier_des_flandres/3c6a5359

...

train ../data/dogbreed/train/a126d857dc3a612bba02ae1d27122d41.jpg ../data/dogbreed/train/sealyham_terrier/a126d857dc3a612bba02ae1d27122d41.jpg
train ../data/dogbreed/train/a198dfcff6e7b12eb4427dbdfe74349e.jpg ../data/dogbreed/train/sealyham_terrier/a198dfcff6e7b12eb4427dbdfe74349e.jpg
train ../data/dogbreed/train/a55866a734b915d94b260d2a5bc642a8.jpg ../data/dogbreed/train/sealyham_terrier/a55866a734b915d94b260d2a5bc642a8.jpg
train ../data/dogbreed/train/a5bf14a3b9f161ce9c458e82d823bea3.jpg ../data/dogbreed/train/sealyham_terrier/a5bf14a3b9f161ce9c458e82d823bea3.jpg
train ../data/dogbreed/train/a63becb08aa9d4c47d5e4a948ae3ddf5.jpg ../data/dogbreed/train/sealyham_terrier/a63becb08aa9d4c47d5e4a948ae3ddf5.jpg
train ../data/dogbreed/train/a705cbd42cf04daaef1961ebb9b50d2b.jpg ../data/dogbreed/train/sealyham_terrier/a705cbd42cf04daaef1961ebb9b50d2b.jpg
train ../data/dogbreed/train/a874515996b484c8cd2864b4ae7fcd85.jpg ../data/dogbreed/train/sealyham_terrier/a874515996b484c8cd2864b4ae7fcd85.jpg

train ../data/dogbreed/train/d26ebabd606703ea526163568193fb4d.jpg ../data/dogbreed/train/soft-coated_wheaten_terrier/d26ebabd606703ea526163568193fb4d.jpg
train ../data/dogbreed/train/d68803c168dd707b81df84496ea2f59b.jpg ../data/dogbreed/train/soft-coated_wheaten_terrier/d68803c168dd707b81df84496ea2f59b.jpg
train ../data/dogbreed/train/deb88665ad11440ea55963b57e9ad79c.jpg ../data/dogbreed/train/soft-coated_wheaten_terrier/deb88665ad11440ea55963b57e9ad79c.jpg
valid ../data/dogbreed/train/df823d73ef4bdeda8b61e3435db0ee6b.jpg ../data/dogbreed/valid/soft-coated_wheaten_terrier/df823d73ef4bdeda8b61e3435db0ee6b.jpg
valid ../data/dogbreed/train/e1efa8c07decd2fe9c807cbfe157b8bf.jpg ../data/dogbreed/valid/soft-coated_wheaten_terrier/e1efa8c07decd2fe9c807cbfe157b8bf.jpg
valid ../data/dogbreed/train/e90868c5948296e603f26b2d2696f9c7.jpg ../data/dogbreed/valid/soft-coated_wheaten_terrier/e90868c5948296e603f26b2d2696f9c7.jpg
valid ../data/dogbreed/train/ee41243497e7715fe3094324b5aceeda.jpg ../data/do

valid ../data/dogbreed/train/f34628fdce6eb2c69ad9c65d0f44f2f7.jpg ../data/dogbreed/valid/tibetan_mastiff/f34628fdce6eb2c69ad9c65d0f44f2f7.jpg
valid ../data/dogbreed/train/fb211e01f77eedac02e92d6bfeedb3ab.jpg ../data/dogbreed/valid/tibetan_mastiff/fb211e01f77eedac02e92d6bfeedb3ab.jpg
valid ../data/dogbreed/train/fd8d47a35e1c84795b4a2a27b761eb7b.jpg ../data/dogbreed/valid/tibetan_mastiff/fd8d47a35e1c84795b4a2a27b761eb7b.jpg
valid ../data/dogbreed/train/fdcf75632c624d8d03b37ddb1c6fc592.jpg ../data/dogbreed/valid/tibetan_mastiff/fdcf75632c624d8d03b37ddb1c6fc592.jpg
valid ../data/dogbreed/train/fe624532170510bd80627c0500bafc97.jpg ../data/dogbreed/valid/tibetan_mastiff/fe624532170510bd80627c0500bafc97.jpg
train ../data/dogbreed/train/013f8fdf6d638c7bb042f5f17e8a9fdc.jpg ../data/dogbreed/train/tibetan_terrier/013f8fdf6d638c7bb042f5f17e8a9fdc.jpg
train ../data/dogbreed/train/07eeb2ef42c6b260684e8cb8e8c4ba70.jpg ../data/dogbreed/train/tibetan_terrier/07eeb2ef42c6b260684e8cb8e8c4ba70.jpg
train 

train ../data/dogbreed/train/5b3f3cad7bcad88cb917df18ece875cb.jpg ../data/dogbreed/train/weimaraner/5b3f3cad7bcad88cb917df18ece875cb.jpg
train ../data/dogbreed/train/61d4222fa93848e18d1e9f39e9644c48.jpg ../data/dogbreed/train/weimaraner/61d4222fa93848e18d1e9f39e9644c48.jpg
train ../data/dogbreed/train/6342835fdefefa74d102b4716a151c88.jpg ../data/dogbreed/train/weimaraner/6342835fdefefa74d102b4716a151c88.jpg
train ../data/dogbreed/train/660e3d56105b467159370bc1bc5e469d.jpg ../data/dogbreed/train/weimaraner/660e3d56105b467159370bc1bc5e469d.jpg
train ../data/dogbreed/train/67013986c89b3e6ba3d9574eb9b0d202.jpg ../data/dogbreed/train/weimaraner/67013986c89b3e6ba3d9574eb9b0d202.jpg
train ../data/dogbreed/train/67bacb6d8fd1b35860764780e47c4e07.jpg ../data/dogbreed/train/weimaraner/67bacb6d8fd1b35860764780e47c4e07.jpg
train ../data/dogbreed/train/6b3ec7c2f0c3101082258b67b9966134.jpg ../data/dogbreed/train/weimaraner/6b3ec7c2f0c3101082258b67b9966134.jpg
train ../data/dogbreed/train/6ca9149d85c7

train ../data/dogbreed/train/153697b802875722cb25421a661a4526.jpg ../data/dogbreed/train/yorkshire_terrier/153697b802875722cb25421a661a4526.jpg
train ../data/dogbreed/train/1548eb783093ac6bdf011d07f3370a36.jpg ../data/dogbreed/train/yorkshire_terrier/1548eb783093ac6bdf011d07f3370a36.jpg
train ../data/dogbreed/train/1574f04538521f284218dfbf2c8678ee.jpg ../data/dogbreed/train/yorkshire_terrier/1574f04538521f284218dfbf2c8678ee.jpg
train ../data/dogbreed/train/1fa82619012d8693282ace6d05e81ccd.jpg ../data/dogbreed/train/yorkshire_terrier/1fa82619012d8693282ace6d05e81ccd.jpg
train ../data/dogbreed/train/23418e1e584a9ac9e51c1ab2c804c1a7.jpg ../data/dogbreed/train/yorkshire_terrier/23418e1e584a9ac9e51c1ab2c804c1a7.jpg
train ../data/dogbreed/train/254b4eb7218cc3e6f18286b020fb61cd.jpg ../data/dogbreed/train/yorkshire_terrier/254b4eb7218cc3e6f18286b020fb61cd.jpg
train ../data/dogbreed/train/25d13706449a0e5bc89077b99366b448.jpg ../data/dogbreed/train/yorkshire_terrier/25d13706449a0e5bc89077b99366b

In [6]:
import numpy as np
import pandas as pd

In [6]:
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.preprocessing import image

In [241]:
df_train = pd.read_csv(DATA_DIR + '/labels.csv')
# print(df_train.head())
# print(df_train.index, 'index')
# print(df_train.columns, 'columns')
# print(df_train.values[0], 'values')
# print(df_train.describe())
# print(df_train.sort_values(by='breed').head(), 'sorted')
# print(df_train.breed.head())
# print(df_train.id.head())

# print(df_train.groupby(['breed','id']).aggregate(lambda x : tuple(x)).head(), 'grouped')
# print(df_train.groupby(['breed']).aggregate(lambda x : tuple(x)).index, 'grouped-index')
# print(df_train.groupby(['breed']).aggregate(lambda x : list(x)).id['beagle'], 'beagle ids')


df_train_grouped = df_train.groupby('breed').aggregate(lambda x : list(x))
df_beagle = df_train_grouped.id['beagle']

TRAIN_AMT = 0.8

for breed in df_train_grouped.index:
    breed_len = len(df_train_grouped.id[breed])
    breed_len_test = np.ceil(breed_len * TRAIN_AMT)
    breed_len_sample = breed_len - breed_len_test
    print(df_train_grouped.id[breed][:2])
    print('%d train and %d samples (%d total) of type %s' % (breed_len_test, breed_len_sample, breed_len, breed))

print(df_train_grouped.index[0])
print(df_beagle[:4])
print(len(df_beagle))


['00ca18751837cd6a22813f8e221f7819', '01268f0007876a692907bda89468184c']
64 train and 16 samples (80 total) of type affenpinscher
['0379145880ad3978f9b80f0dc2c03fba', '059a86d50634c78d5a18f918cb0b3f0d']
93 train and 23 samples (116 total) of type afghan_hound
['00792e341f3c6eb33663e415d0715370', '014c2b0cd8e3b517e649cecf8543b8fe']
69 train and 17 samples (86 total) of type african_hunting_dog
['0267c5f1acbab52ae4a7927e0398612b', '02c90d8109d9a48739b9887349d92b1f']
86 train and 21 samples (107 total) of type airedale
['02f3d5fd1eea0a2e6f4742bc54b51ba9', '041d0d6a8d110b35a3795dd5c68f9a36']
60 train and 14 samples (74 total) of type american_staffordshire_terrier
['01a41985ab8209e750588fa348d635be', '01e4d3d1cb704ed819bce35d22e716a7']
63 train and 15 samples (78 total) of type appenzeller
['0192d7174233ebdb3d720000413829ac', '05c128c8e3ef0c2739f181f9c5677f56']
82 train and 20 samples (102 total) of type australian_terrier
['003df8b8a8b05244b1d920bb6cf451f9', '04b5834c3a969c6308ce157c547bb

In [232]:
xs = np.arange(10)
print(xs[:8], xs[8:])

[0 1 2 3 4 5 6 7] [8 9]


In [247]:
np.ceil(34.2).astype('int')


35